In [5]:
import copy
import json
from tqdm import tqdm
import os
import re
import math
from collections import defaultdict
from collections import Counter
from random import randrange

from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])

stoplist = open('/Users/liyuetian/Documents/College/NEU/CS/CS6200/HW_Data/IR_data/AP_DATA/stoplist.txt')

stop_arr = []
for line in stoplist:
    stop_arr.append(line.strip())
stop_arr = stop_arr + ['document', 'discuss', 'report', 'include', 'describe', 'identify', 'cite', 'predict', 'taken']




In [2]:
querylist = open('/Users/liyuetian/Documents/College/NEU/CS/CS6200/HW_Data/IR_data/AP_DATA/query_desc.51-100.short.txt')
query_dict = {}
word_arr = []
query_arr = []

for line in querylist:
    if line.strip() != '':
        queryno = re.sub('[^A-Za-z0-9]+', '', line.split()[0])
        
        modified = line[line.find(".")+1:]
        terms = es.indices.analyze(index='hw6', body={'analyzer': 'stopped', 'text': modified})
        for term in terms['tokens']:
            query_arr.append(term['token'])
        for word in query_arr:
            if word not in stop_arr:
                word_arr.append(word)
        query_dict[queryno] = word_arr
        word_arr = []
        query_arr = []
print(query_dict)



{'85': ['alleg', 'measur', 'corrupt', 'public', 'offici', 'government', 'jurisdict', 'worldwid'], '59': ['type', 'weather', 'event', 'directli', 'caus', 'least', 'fatal', 'locat'], '56': ['prime', 'lend', 'rate', 'actual', 'prime', 'rate', 'move'], '71': ['incurs', 'land', 'air', 'water', 'border', 'area', 'countri', 'militari', 'forc', 'second', 'countri', 'guerrilla', 'group', 'base', 'second', 'countri'], '64': ['event', 'result', 'polit', 'motiv', 'hostag', 'take'], '62': ['militari', 'coup', "d'etat", 'attempt', 'success', 'countri'], '93': ['support', 'nation', 'rifl', 'associ', 'nra', 'asset'], '99': ['develop', 'iran', 'contra', 'affair'], '58': ['anticip', 'rail', 'strike', 'ongo', 'rail', 'strike'], '77': ['poach', 'method', 'type', 'wildlif'], '54': ['sign', 'contract', 'preliminari', 'agreement', 'make', 'tent', 'reserv', 'launch', 'commerci', 'satellit'], '87': ['current', 'crimin', 'action', 'offic', 'fail', 'u.', 'financi', 'institut'], '94': ['crime', 'perpetr', 'aid', 

In [3]:
def get_random_train_test():
    test_set = set()
    train_set = set()
    while len(test_set) < 5:
        test_set.add(randrange(1, 26))
    for i in range(1, 26):
        if i not in test_set:
            train_set.add(i)
    qno_map = {1:85, 2:59, 3:56, 4:71, 5:64, 6:62, 7:93, 8:99, 9:58, 10:77, 11:54, 12:87, 13:94, 14:100, 15:89, 16:61, 17:95, 18:68, 19:57, 20:97, 21:98, 22:60, 23:80, 24:63, 25:91}
    return [str(qno_map[i]) for i in test_set], [ str(qno_map[i]) for i in train_set ]



In [6]:
test_queries, train_queries = get_random_train_test()

In [9]:
qrel_dict = {}
with open('./info/qrel_dict.txt', 'r') as f:
    qrel_dict = json.load(f)
    
qrel_dict_all = {}
with open('./info/qrel_dict_all.txt', 'r') as f:
    qrel_dict_all = json.load(f)
    


In [10]:
query_train = {}
query_test = {}

qrel_train = {}
for q in train_queries:
    qrel_train[q] = qrel_dict[q]
    query_train[q] = query_dict[q]
    
qrel_test = {}
for q in test_queries:
    qrel_test[q] = qrel_dict_all[q]
    query_test[q] = query_dict[q]
    

In [36]:
with open('./info/qrel_train.txt', 'w') as f:
    json.dump(qrel_train, f)

with open('./info/qrel_test.txt', 'w') as f:
    json.dump(qrel_test, f)

with open('./info/query_train.txt', 'w') as f:
    json.dump(query_train, f)

with open('./info/query_test.txt', 'w') as f:
    json.dump(query_test, f)



In [11]:
tf_info = []
with open('./info/tf_info.txt', 'r') as f:
    tf_info = json.load(f)

doc_len_info = {}
with open('./info/doc_len_info.txt', 'r') as f:
    doc_len_info = json.load(f)

def vocab_size():
    search_result = es.search(index='ap_data', size=0, body={
            "aggs" : {
                "unique_terms" : {
                    "cardinality" : {"field": "text"}
                }
            }
        }
    )
    return int(search_result['aggregations']['unique_terms']['value'])

voc_size = vocab_size()

        
def tf_q(queryno, term):
    count = Counter(query_dict[queryno])
    return count[term]

def field_stat():
    return es.termvectors(index='ap_data', id='AP890101-0001', fields='text')['term_vectors']['text']['field_statistics']

# Get the total number of docs in the corpus
total_docs = es.count(index='ap_data')['count']

# Get the averge length of th e docs
avg_len = field_stat()['sum_ttf'] / total_docs

In [31]:
def okapi_tf(tf, doc_len):
    return tf / (tf + 0.5 + 1.5 * (doc_len/avg_len))

def tf_idf(tf, doc_len, df, total_docs):
    if df == 0:
        return 0
    else:
        return okapi_tf(tf, doc_len) * math.log(total_docs / df)

def okapi_bm25(tf, tf_q, doc_len, df, k1, k2, b, total_docs, avg_len):
    return math.log((total_docs + 0.5) / (df + 0.5)) * \
           ((tf + k1 * tf) / (tf + k1* ((1-b) + b * (doc_len / avg_len)))) * \
           ((tf_q + k2 * tf_q) / (tf_q + k2))

def unigram_lm_laplace(tf, doc_len, voc_size):
    return math.log((tf + 1) / (doc_len + voc_size))

def unigram_lm_jm(tf, doc_len, all_tf, all_len, lmbda):
    frntgrd = 0
    if doc_len != 0:
        frntgrd = float(tf) / doc_len
    backgrd = 0
    if (all_len - doc_len) != 0:
        backgrd = float(all_tf - tf) / (all_len - doc_len)
    return lmbda * frntgrd + (1 - lmbda) * backgrd

def rank_scores(key, scores, out):
    sorted_scores = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    for j in range(len(scores)):
        str = ('{} Q0 {} {} {} Exp'
            .format(key, sorted_scores[j][0], j+1, sorted_scores[j][1]))
        out.write(str+"\n")
    scores.clear()

def add_score(docno, model_scores, score):
    if docno in model_scores:
        model_scores[docno] += score
    else:
        model_scores[docno] = score
        

In [13]:
def compute_tf_okapi_tf(query, qrel_dict, out_path):
    okapi_tf_out = open(out_path, "w")

    okapi_tf_scores = defaultdict(lambda: 0.0)

    for key, words in tqdm(query.items(), position=0, leave=True, desc='Computing Model'):
        for word in words:
            new_tf_info = [element for element in tf_info if element['word'] == word]
            docs = qrel_dict[key]
            for docno in docs:
                tf = 0
                for item in new_tf_info:
                    if item['id'] == docno:
                        tf = item['tf']
                        break
#                 df = len(new_tf_info)
#                 qf = tf_q(key, word)
                doc_len = doc_len_info[docno]
                add_score(docno, okapi_tf_scores, okapi_tf(tf, doc_len))
        rank_scores(key, okapi_tf_scores, okapi_tf_out)
    okapi_tf_out.close()



In [37]:
def compute_tf_idf(query, qrel_dict, out_path):
    tf_idf_out = open(out_path, "w")

    tf_idf_scores = defaultdict(lambda: 0.0)

    for key, words in tqdm(query.items(), position=0, leave=True, desc='Computing Models'):
        for word in words:
            new_tf_info = [element for element in tf_info if element['word'] == word]
            docs = qrel_dict[key]
            df = len(new_tf_info)
            for docno in docs:
                tf = 0
                for item in new_tf_info:
                    if item['id'] == docno:
                        tf = item['tf']
                        break
#                 qf = tf_q(key, word)
                doc_len = doc_len_info[docno]
                add_score(docno, tf_idf_scores, tf_idf(tf, doc_len, df, total_docs))
        rank_scores(key, tf_idf_scores, tf_idf_out)
    tf_idf_out.close()



In [15]:
def compute_bm25(query, qrel_dict, out_path):
    okapi_bm25_out = open(out_path, "w")

    okapi_bm25_scores = defaultdict(lambda: 0.0)

    for key, words in tqdm(query.items(), position=0, leave=True, desc='Computing Models'):
        for word in words:
            new_tf_info = [element for element in tf_info if element['word'] == word]
            docs = qrel_dict[key]
            df = len(new_tf_info)
            for docno in docs:
                tf = 0
                for item in new_tf_info:
                    if item['id'] == docno:
                        tf = item['tf']
                        break
                
                qf = tf_q(key, word)
                doc_len = doc_len_info[docno]
                add_score(docno, okapi_bm25_scores, okapi_bm25(tf, qf, doc_len, df, 1.2, 100, 0.75, total_docs, avg_len))
        rank_scores(key, okapi_bm25_scores, okapi_bm25_out)
    okapi_bm25_out.close()

In [16]:
def compute_unigram_laplace(query, qrel_dict, out_path):
    laplace_scores = defaultdict(lambda: 0.0)
    out = open(out_path, "w")
#     voc_size = vocab_size()
    for key,words in tqdm(query.items(), position=0, leave=True, desc='Computing Unigram LM Laplace Model'):
        for word in words:
            new_tf_info = [element for element in tf_info if element['word'] == word]
            docs = qrel_dict[key]
            for docno in docs:
                tf = 0
                for item in new_tf_info:
                    if item['id'] == docno:
                        tf = item['tf']
                        break
                doc_len = doc_len_info[docno]
                score = unigram_lm_laplace(tf, doc_len, voc_size)
                add_score(docno, laplace_scores, score)
        rank_scores(key, laplace_scores, out)
    out.close()

In [33]:
def compute_unigram_jm(query, qrel_dict, out_path):
    jelinek_scores = defaultdict(lambda: 0.0)
    out = open(out_path, "w")
    lmbda = .7
    for key,words in tqdm(query.items(), position=0, leave=True, desc='Computing Unigram LM JM Model'):
        for word in words:
            new_tf_info = [element for element in tf_info if element['word'] == word]
            docs = qrel_dict[key]
            all_tf = 0
            all_len = 0
            for item in new_tf_info:
                if item['word'] == word:
                    tf = item['tf']
                    all_tf = all_tf + tf
                    all_len = all_len + doc_len_info[item['id']]
            for docno in docs:
                tf = 0
                for item in new_tf_info:
                    if item['id'] == docno:
                        tf = item['tf']
                        break
                doc_len = doc_len_info[docno]
                lmbda = .7
                score = unigram_lm_jm(tf, doc_len, all_tf, all_len, lmbda)
                add_score(docno, jelinek_scores, score)
        rank_scores(key, jelinek_scores, out)
    out.close()

# Compute training matrix

In [19]:
compute_tf_okapi_tf(query_train, qrel_train, './result/okapi_tf_train.txt')

Computing Model: 100%|██████████| 20/20 [03:00<00:00,  9.00s/it]


In [38]:
compute_tf_idf(query_train, qrel_train, './result/tf_idf_train.txt')

Computing Models: 100%|██████████| 20/20 [02:06<00:00,  6.34s/it]


In [22]:
compute_bm25(query_train, qrel_train, './result/bm25_train.txt')

Computing Models: 100%|██████████| 20/20 [03:23<00:00, 10.15s/it]


In [23]:
compute_unigram_laplace(query_train, qrel_train, './result/unigram_laplace_train.txt')

Computing Unigram LM Laplace Model: 100%|██████████| 20/20 [02:22<00:00,  7.14s/it]


In [24]:
compute_unigram_jm(query_train, qrel_train, './result/unigram_jm_train.txt')

Computing Unigram LM JM Model: 100%|██████████| 20/20 [02:06<00:00,  6.34s/it]


# Compute testing matrix

In [25]:
compute_tf_okapi_tf(query_test, qrel_test, './result/okapi_tf_test.txt')

Computing Model: 100%|██████████| 5/5 [31:29<00:00, 377.98s/it]


In [26]:
compute_tf_idf(query_test, qrel_test, './result/tf_idf_test.txt')

Computing Models: 100%|██████████| 5/5 [31:30<00:00, 378.01s/it]


In [27]:
compute_bm25(query_test, qrel_test, './result/bm25_test.txt')

Computing Models: 100%|██████████| 5/5 [32:16<00:00, 387.29s/it]


In [28]:
compute_unigram_laplace(query_test, qrel_test, './result/unigram_laplace_test.txt')

Computing Unigram LM Laplace Model: 100%|██████████| 5/5 [31:29<00:00, 377.89s/it]


In [34]:
compute_unigram_jm(query_test, qrel_test, './result/unigram_jm_test.txt')

Computing Unigram LM JM Model: 100%|██████████| 5/5 [32:32<00:00, 390.56s/it]
